## Importing Dataset

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread
import cv2
import random
from os import listdir
from sklearn.preprocessing import  LabelBinarizer
from keras.preprocessing import image
from keras_preprocessing.image import img_to_array, array_to_img
from keras.optimizers import Adam
from PIL import Image
from keras.utils import to_categorical
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D,Activation, Flatten, Dropout, Dense, LeakyReLU, ReLU
from sklearn.model_selection import train_test_split

In [2]:
#Converting Images to array 
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, (256,256))  
            #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [3]:
dir = "Plant_images_pianalytix"
root_dir = listdir(dir)
image_list, label_list = [], []
all_labels = ['Corn-Common_rust', 'Potato-Early_blight', 'Tomato-Bacterial_spot']
binary_labels = [0,1,2]
temp = -1

# Reading and converting image to numpy array
for directory in root_dir:
  plant_image_list = listdir(f"{dir}/{directory}")
  temp += 1
  for files in plant_image_list:
    image_path = f"{dir}/{directory}/{files}"
    image_list.append(convert_image_to_array(image_path))
    label_list.append(binary_labels[temp])

In [4]:
# Visualize the number of classes count
label_counts = pd.DataFrame(label_list).value_counts()
label_counts.head()

0    300
1    300
2    300
dtype: int64

In [5]:
# checking ther sahpe of the image
image_list[0].shape

(256, 256, 3)

In [6]:
# Checking teh length of the label list
label_list = np.array(label_list)
label_list.shape

(900,)

In [7]:
# Splitting dataset into test and train
x_train, x_test, y_train, y_test = train_test_split(image_list, label_list, test_size=0.2, random_state = 10) 

### Normalizing the Datasets

In [8]:
# Normalize and reshape data
x_train = np.array(x_train, dtype=np.float16) / 225.0
x_test = np.array(x_test, dtype=np.float16) / 225.0
x_train = x_train.reshape( -1, 256,256,3)
x_test = x_test.reshape( -1, 256,256,3)

In [9]:
## One-hot encoding the target variable
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [10]:
# Splitting the training data set into training and validation data sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2)

## Applying CNN Model

In [11]:
model = Sequential()

model.add(Conv2D(filters=16, kernel_size=2, input_shape=(256,256,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=2))


model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(3,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 16)      208       
                                                                 
 activation (Activation)     (None, 256, 256, 16)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      2080      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 131072)            0

In [12]:
# Compiling model
model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop',metrics=['accuracy'])

In [13]:
# Training the model
epochs = 12
batch_size = 32

# Early Stopping
callback = EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_data = (x_val, y_val), callbacks = [callback])

Epoch 1/12
18/18 [==============================] - 11s 583ms/step - loss: 2.7420 - accuracy: 0.7309 - val_loss: 0.1137 - val_accuracy: 0.9653
Epoch 2/12
18/18 [==============================] - 11s 624ms/step - loss: 0.1249 - accuracy: 0.9514 - val_loss: 0.0712 - val_accuracy: 0.9792
Epoch 3/12
18/18 [==============================] - 12s 698ms/step - loss: 0.2245 - accuracy: 0.9340 - val_loss: 0.0366 - val_accuracy: 0.9931
Epoch 4/12
18/18 [==============================] - 15s 837ms/step - loss: 0.0049 - accuracy: 1.0000 - val_loss: 0.0328 - val_accuracy: 0.9931
Epoch 5/12
18/18 [==============================] - 14s 795ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.0340 - val_accuracy: 0.9931
Epoch 6/12
18/18 [==============================] - 13s 738ms/step - loss: 4.8239e-04 - accuracy: 1.0000 - val_loss: 0.0399 - val_accuracy: 0.9931
Epoch 7/12
18/18 [==============================] - 14s 788ms/step - loss: 0.2813 - accuracy: 0.9618 - val_loss: 0.0369 - val_accuracy: 0.

In [14]:
# Calculating test accuracy 
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100} %")

6/6 [==============================] - 1s 149ms/step - loss: 0.0345 - accuracy: 0.9889
Test Accuracy: 98.88888597488403 %


## Applying CNN with Different Activation Functions

### CNN with ELU

In [15]:
## Creating a simple CNN architecture with three convolutional layer, each followed by a max pooling layer

model = Sequential()

model.add(Conv2D(filters=16, kernel_size=2, input_shape=(256,256,3), padding='same'))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=32, kernel_size=2, activation='elu', padding='same'))
model.add(MaxPooling2D(pool_size=2))


model.add(Flatten())
model.add(Dense(128))
model.add(Activation('elu'))
model.add(Dense(3,activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 256, 256, 16)      208       
                                                                 
 activation_2 (Activation)   (None, 256, 256, 16)      0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 128, 128, 16)     0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 128, 128, 32)      2080      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 131072)           

In [16]:
# Compiling model
model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop',metrics=['accuracy'])

In [17]:
# Training the model
epochs = 12
batch_size = 32

# Early Stopping
callback = EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_data = (x_val, y_val), callbacks = [callback])

Epoch 1/12
18/18 [==============================] - 14s 766ms/step - loss: 10.2713 - accuracy: 0.7031 - val_loss: 4.0071 - val_accuracy: 0.6806
Epoch 2/12
18/18 [==============================] - 14s 788ms/step - loss: 0.8978 - accuracy: 0.8507 - val_loss: 0.0560 - val_accuracy: 0.9931
Epoch 3/12
18/18 [==============================] - 17s 923ms/step - loss: 0.3147 - accuracy: 0.9444 - val_loss: 0.0439 - val_accuracy: 0.9861
Epoch 4/12
18/18 [==============================] - 14s 783ms/step - loss: 0.5439 - accuracy: 0.9097 - val_loss: 0.1249 - val_accuracy: 0.9861
Epoch 5/12
18/18 [==============================] - 15s 862ms/step - loss: 0.0081 - accuracy: 1.0000 - val_loss: 0.1158 - val_accuracy: 0.9861
Epoch 6/12
18/18 [==============================] - 14s 767ms/step - loss: 0.3479 - accuracy: 0.9080 - val_loss: 0.0547 - val_accuracy: 0.9861


In [18]:
# Calculating test accuracy 
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100} %")

6/6 [==============================] - 1s 169ms/step - loss: 0.0600 - accuracy: 0.9722
Test Accuracy: 97.22222089767456 %


### CNN with selu

In [19]:
## Creating a simple CNN architecture with three convolutional layer, each followed by a max pooling layer

model = Sequential()

model.add(Conv2D(filters=16, kernel_size=2, input_shape=(256,256,3), padding='same'))
model.add(Activation('selu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=32, kernel_size=2, activation='selu', padding='same'))
model.add(MaxPooling2D(pool_size=2))


model.add(Flatten())
model.add(Dense(128))
model.add(Activation('selu'))
model.add(Dense(3,activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 256, 256, 16)      208       
                                                                 
 activation_4 (Activation)   (None, 256, 256, 16)      0         
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 128, 128, 16)     0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 128, 128, 32)      2080      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 131072)           

In [20]:
# Compiling model
model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop',metrics=['accuracy'])

In [21]:
# Training the model
epochs = 12
batch_size = 32

# Early Stopping
callback = EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_data = (x_val, y_val), callbacks = [callback])

Epoch 1/12
18/18 [==============================] - 18s 939ms/step - loss: 21.2330 - accuracy: 0.6111 - val_loss: 0.2773 - val_accuracy: 0.9236
Epoch 2/12
18/18 [==============================] - 16s 907ms/step - loss: 0.3019 - accuracy: 0.9288 - val_loss: 0.2815 - val_accuracy: 0.9375
Epoch 3/12
18/18 [==============================] - 17s 927ms/step - loss: 0.1205 - accuracy: 0.9514 - val_loss: 0.3493 - val_accuracy: 0.8819
Epoch 4/12
18/18 [==============================] - 15s 816ms/step - loss: 0.5466 - accuracy: 0.8767 - val_loss: 0.8096 - val_accuracy: 0.7361


In [22]:
# Calculating test accuracy 
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100} %")

6/6 [==============================] - 1s 146ms/step - loss: 0.7461 - accuracy: 0.7222
Test Accuracy: 72.22222089767456 %


## Using Average Pooling between Layers

### CNN with Relu

In [23]:
## Creating a simple CNN architecture with three convolutional layer, each followed by a average pooling layer

model = Sequential()

model.add(Conv2D(filters=16, kernel_size=2, input_shape=(256,256,3), padding='same'))
model.add(Activation('relu'))
model.add(AveragePooling2D(pool_size=2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu', padding='same'))
model.add(AveragePooling2D(pool_size=2))


model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(3,activation='softmax'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 256, 256, 16)      208       
                                                                 
 activation_6 (Activation)   (None, 256, 256, 16)      0         
                                                                 
 average_pooling2d (AverageP  (None, 128, 128, 16)     0         
 ooling2D)                                                       
                                                                 
 conv2d_7 (Conv2D)           (None, 128, 128, 32)      2080      
                                                                 
 average_pooling2d_1 (Averag  (None, 64, 64, 32)       0         
 ePooling2D)                                                     
                                                                 
 flatten_3 (Flatten)         (None, 131072)           

In [24]:
# Compiling model
model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop',metrics=['accuracy'])

In [25]:
# Training the model
epochs = 12
batch_size = 32

# Early Stopping
callback = EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_data = (x_val, y_val), callbacks = [callback])

Epoch 1/12
18/18 [==============================] - 10s 548ms/step - loss: 3.7573 - accuracy: 0.6979 - val_loss: 0.2006 - val_accuracy: 0.9167
Epoch 2/12
18/18 [==============================] - 9s 524ms/step - loss: 0.5716 - accuracy: 0.8663 - val_loss: 0.3109 - val_accuracy: 0.9097
Epoch 3/12
18/18 [==============================] - 10s 537ms/step - loss: 0.3361 - accuracy: 0.9358 - val_loss: 0.0685 - val_accuracy: 0.9722
Epoch 4/12
18/18 [==============================] - 10s 585ms/step - loss: 0.0431 - accuracy: 0.9913 - val_loss: 0.0694 - val_accuracy: 0.9722
Epoch 5/12
18/18 [==============================] - 11s 612ms/step - loss: 0.1090 - accuracy: 0.9705 - val_loss: 0.0830 - val_accuracy: 0.9722
Epoch 6/12
18/18 [==============================] - 11s 632ms/step - loss: 0.0119 - accuracy: 0.9965 - val_loss: 0.0467 - val_accuracy: 0.9861
Epoch 7/12
18/18 [==============================] - 11s 632ms/step - loss: 0.0955 - accuracy: 0.9792 - val_loss: 0.0423 - val_accuracy: 0.9861


In [26]:
# Calculating test accuracy 
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100} %")

6/6 [==============================] - 1s 168ms/step - loss: 0.0077 - accuracy: 1.0000
Test Accuracy: 100.0 %


### CNN with ELU

In [27]:
## Creating a simple CNN architecture with three convolutional layer, each followed by a average pooling layer

model = Sequential()

model.add(Conv2D(filters=16, kernel_size=2, input_shape=(256,256,3), padding='same'))
model.add(Activation('elu'))
model.add(AveragePooling2D(pool_size=2))

model.add(Conv2D(filters=32, kernel_size=2, activation='elu', padding='same'))
model.add(AveragePooling2D(pool_size=2))


model.add(Flatten())
model.add(Dense(128))
model.add(Activation('elu'))
model.add(Dense(3,activation='softmax'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 256, 256, 16)      208       
                                                                 
 activation_8 (Activation)   (None, 256, 256, 16)      0         
                                                                 
 average_pooling2d_2 (Averag  (None, 128, 128, 16)     0         
 ePooling2D)                                                     
                                                                 
 conv2d_9 (Conv2D)           (None, 128, 128, 32)      2080      
                                                                 
 average_pooling2d_3 (Averag  (None, 64, 64, 32)       0         
 ePooling2D)                                                     
                                                                 
 flatten_4 (Flatten)         (None, 131072)           

In [28]:
# Compiling model
model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop',metrics=['accuracy'])

In [29]:
# Training the model
epochs = 12
batch_size = 32

# Early Stopping
callback = EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_data = (x_val, y_val), callbacks = [callback])

Epoch 1/12
18/18 [==============================] - 15s 802ms/step - loss: 15.0437 - accuracy: 0.5608 - val_loss: 0.9431 - val_accuracy: 0.7153
Epoch 2/12
18/18 [==============================] - 14s 775ms/step - loss: 0.7397 - accuracy: 0.8125 - val_loss: 4.6205 - val_accuracy: 0.3056
Epoch 3/12
18/18 [==============================] - 13s 739ms/step - loss: 0.4305 - accuracy: 0.8837 - val_loss: 0.0489 - val_accuracy: 0.9861
Epoch 4/12
18/18 [==============================] - 14s 783ms/step - loss: 0.0317 - accuracy: 0.9913 - val_loss: 0.0522 - val_accuracy: 0.9792
Epoch 5/12
18/18 [==============================] - 13s 703ms/step - loss: 6.4322 - accuracy: 0.8247 - val_loss: 0.0918 - val_accuracy: 0.9792
Epoch 6/12
18/18 [==============================] - 11s 618ms/step - loss: 0.0225 - accuracy: 0.9931 - val_loss: 0.0409 - val_accuracy: 0.9792
Epoch 7/12
18/18 [==============================] - 11s 597ms/step - loss: 0.1052 - accuracy: 0.9618 - val_loss: 0.4577 - val_accuracy: 0.847

In [30]:
# Calculating test accuracy 
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100} %")

6/6 [==============================] - 1s 218ms/step - loss: 0.0248 - accuracy: 0.9889
Test Accuracy: 98.88888597488403 %


### CNN with SELU

In [31]:
## Creating a simple CNN architecture with three convolutional layer, each followed by a average pooling layer

model = Sequential()

model.add(Conv2D(filters=16, kernel_size=2, input_shape=(256,256,3), padding='same'))
model.add(Activation('selu'))
model.add(AveragePooling2D(pool_size=2))

model.add(Conv2D(filters=32, kernel_size=2, activation='selu', padding='same'))
model.add(AveragePooling2D(pool_size=2))


model.add(Flatten())
model.add(Dense(128))
model.add(Activation('selu'))
model.add(Dense(3,activation='softmax'))
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 256, 256, 16)      208       
                                                                 
 activation_10 (Activation)  (None, 256, 256, 16)      0         
                                                                 
 average_pooling2d_4 (Averag  (None, 128, 128, 16)     0         
 ePooling2D)                                                     
                                                                 
 conv2d_11 (Conv2D)          (None, 128, 128, 32)      2080      
                                                                 
 average_pooling2d_5 (Averag  (None, 64, 64, 32)       0         
 ePooling2D)                                                     
                                                                 
 flatten_5 (Flatten)         (None, 131072)           

In [32]:
# Compiling model
model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop',metrics=['accuracy'])

In [33]:
# Training the model
epochs = 12
batch_size = 32

# Early Stopping
callback = EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_data = (x_val, y_val), callbacks = [callback])

Epoch 1/12
18/18 [==============================] - 14s 726ms/step - loss: 44.4798 - accuracy: 0.3733 - val_loss: 0.6455 - val_accuracy: 0.8125
Epoch 2/12
18/18 [==============================] - 12s 663ms/step - loss: 1.3000 - accuracy: 0.7292 - val_loss: 0.2641 - val_accuracy: 0.9306
Epoch 3/12
18/18 [==============================] - 12s 648ms/step - loss: 0.4836 - accuracy: 0.8299 - val_loss: 0.1476 - val_accuracy: 0.9444
Epoch 4/12
18/18 [==============================] - 11s 622ms/step - loss: 0.3996 - accuracy: 0.8403 - val_loss: 0.0973 - val_accuracy: 0.9653
Epoch 5/12
18/18 [==============================] - 12s 648ms/step - loss: 0.3616 - accuracy: 0.8872 - val_loss: 3.7522 - val_accuracy: 0.6528
Epoch 6/12
18/18 [==============================] - 17s 948ms/step - loss: 1.3445 - accuracy: 0.8646 - val_loss: 0.1107 - val_accuracy: 0.9583
Epoch 7/12
18/18 [==============================] - 16s 896ms/step - loss: 0.0594 - accuracy: 0.9878 - val_loss: 0.0259 - val_accuracy: 0.993

In [34]:
# Calculating test accuracy 
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100} %")

6/6 [==============================] - 1s 195ms/step - loss: 0.1950 - accuracy: 0.9556
Test Accuracy: 95.55555582046509 %


## Using different Architecture

### LeNet Architecture

In [59]:
model = Sequential()

model.add(Conv2D(filters=6, kernel_size=(3, 3), activation='relu', input_shape=(256,256,3)))
model.add(AveragePooling2D())

model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D())

model.add(Flatten())
model.add(Dense(units=120, activation='relu'))
model.add(Dense(units=84, activation='relu'))
model.add(Dense(3, activation = 'softmax'))
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 254, 254, 6)       168       
                                                                 
 average_pooling2d_12 (Avera  (None, 127, 127, 6)      0         
 gePooling2D)                                                    
                                                                 
 conv2d_25 (Conv2D)          (None, 125, 125, 16)      880       
                                                                 
 average_pooling2d_13 (Avera  (None, 62, 62, 16)       0         
 gePooling2D)                                                    
                                                                 
 flatten_12 (Flatten)        (None, 61504)             0         
                                                                 
 dense_24 (Dense)            (None, 120)             

In [60]:
# Compiling model
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam',metrics=['accuracy'])

In [61]:
# Training the model
epochs = 50
batch_size = 32

# Early Stopping
callback = EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_data = (x_val, y_val), callbacks = [callback])

Epoch 1/50
18/18 [==============================] - 10s 479ms/step - loss: 0.6747 - accuracy: 0.7205 - val_loss: 0.2241 - val_accuracy: 0.9514
Epoch 2/50
18/18 [==============================] - 8s 441ms/step - loss: 0.1391 - accuracy: 0.9583 - val_loss: 0.1316 - val_accuracy: 0.9514
Epoch 3/50
18/18 [==============================] - 8s 443ms/step - loss: 0.0647 - accuracy: 0.9774 - val_loss: 0.0617 - val_accuracy: 0.9861
Epoch 4/50
18/18 [==============================] - 8s 441ms/step - loss: 0.0207 - accuracy: 0.9965 - val_loss: 0.0533 - val_accuracy: 0.9861
Epoch 5/50
18/18 [==============================] - 8s 443ms/step - loss: 0.0087 - accuracy: 0.9983 - val_loss: 0.0481 - val_accuracy: 0.9861
Epoch 6/50
18/18 [==============================] - 8s 440ms/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 0.0525 - val_accuracy: 0.9861
Epoch 7/50
18/18 [==============================] - 8s 436ms/step - loss: 8.1254e-04 - accuracy: 1.0000 - val_loss: 0.0644 - val_accuracy: 0.9861
E

In [62]:
# Calculating test accuracy 
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100} %")

6/6 [==============================] - 1s 120ms/step - loss: 0.0195 - accuracy: 0.9833
Test Accuracy: 98.33333492279053 %


### AlexNet Architecture

In [63]:
model = Sequential()

model.add(Conv2D(filters=96, kernel_size=(11, 11), strides=(4, 4), activation="relu", input_shape=(256,256, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3, 3), strides= (2, 2)))

model.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

model.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation="relu", padding="same"))
model.add(BatchNormalization())

model.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation="relu", padding="same"))
model.add(BatchNormalization())

model.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

model.add(Flatten())
model.add(Dense(4096, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(3, activation="softmax"))

In [64]:
# Compiling model
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam',metrics=['accuracy'])

In [65]:
# Training the model
epochs = 12
batch_size = 32

# Early Stopping
callback = EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_data = (x_val, y_val), callbacks = [callback])

Epoch 1/12
18/18 [==============================] - 37s 2s/step - loss: 4.5139 - accuracy: 0.8646 - val_loss: 2795.9429 - val_accuracy: 0.3056
Epoch 2/12
18/18 [==============================] - 36s 2s/step - loss: 4.5631 - accuracy: 0.9566 - val_loss: 2326.1111 - val_accuracy: 0.3056
Epoch 3/12
18/18 [==============================] - 37s 2s/step - loss: 1.2404 - accuracy: 0.9635 - val_loss: 324.9420 - val_accuracy: 0.4375
Epoch 4/12
18/18 [==============================] - 36s 2s/step - loss: 1.6752 - accuracy: 0.9740 - val_loss: 111.5938 - val_accuracy: 0.3750
Epoch 5/12
18/18 [==============================] - 36s 2s/step - loss: 0.7728 - accuracy: 0.9774 - val_loss: 73.7565 - val_accuracy: 0.5069
Epoch 6/12
18/18 [==============================] - 36s 2s/step - loss: 0.4191 - accuracy: 0.9809 - val_loss: 5.9523 - val_accuracy: 0.7778
Epoch 7/12
18/18 [==============================] - 36s 2s/step - loss: 0.3432 - accuracy: 0.9809 - val_loss: 17.5964 - val_accuracy: 0.6667
Epoch 8/

In [66]:
# Calculating test accuracy 
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100} %")

6/6 [==============================] - 2s 406ms/step - loss: 78.6642 - accuracy: 0.6556
Test Accuracy: 65.55555462837219 %
